In [2]:
!pip install langchain faiss-cpu openai datasets

# for basic RAG section
!pip install --upgrade langchain
!pip install --upgrade langchain-community
!pip install --upgrade transformers
!pip install --upgrade sentence-transformers
!pip install --upgrade faiss-cpu
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.5 MB/s eta 0:00:00
   ━

In [15]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

Overview:

- Blackbox PoisonedRAG Attacks, like the paper

- NQ Benchmark, 100 examples like the paper

### Load NQ Benchmark

too big, don't use

In [ ]:
# This here loads the entire dataset and splits 100 examples from the train. I'm just going to stream.

# from datasets import load_dataset

# nq_dataset = load_dataset("natural_questions", split="train[:100]")  # Use a subset for demo

# questions = [{"id": idx, "question": item["question_text"]} for idx, item in enumerate(nq_dataset)]
# print(f"Loaded {len(questions)} questions.")




# This here loads 100 streaming.
# from datasets import load_dataset

# nq_dataset = load_dataset("natural_questions", split="train", streaming=True)
# limited_questions = [item for _, item in zip(range(100), nq_dataset)]
# print(f"Loaded {len(limited_questions)} questions.")

In [ ]:
# # Load Wikipedia dataset
# wikipedia_dataset = load_dataset("wikipedia", "20220301.en", split="train[:10000]")  # Demo subset

# # Extract passages
# passages = [{"text": item["text"], "title": item["title"]} for item in wikipedia_dataset]
# print(f"Loaded {len(passages)} Wikipedia passages.")


### Load SQUAD Benchmark

a lot smaller

In [5]:
from datasets import load_dataset
squad = load_dataset("squad", split="train[:100]")

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [6]:
len(squad)

# number of contexts is len(squad) / 5

100

In [7]:
from collections import defaultdict

def add_line_breaks(text, line_length=200):
    """Add line breaks to text every specified number of characters."""
    return '\n'.join(text[i:i+line_length] for i in range(0, len(text), line_length))


context_questions = defaultdict(list)

# Group questions by their context
for item in squad:
    context = add_line_breaks(item["context"])
    question = item["question"]
    answers = item["answers"]["text"]

    context_questions[context].append({
        "question": question,
        "answers": answers
    })

# Print unique contexts with their related questions
print(f"Total unique contexts: {len(context_questions)}\n")

for context, questions in context_questions.items():
    print("=" * 80)
    print(f"Context:\n{context}\n")
    print(f"Related Questions ({len(questions)}):")
    for q_idx, q_data in enumerate(questions, 1):
        print(f"{q_idx}. Question: {q_data['question']}")
        print(f"   Answers: {q_data['answers']}\n")

Total unique contexts: 21

Context:
Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper sta
tue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of p
rayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct lin
e that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.

Related Questions (5):
1. Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
   Answers: ['Saint Bernadette Soubirous']

2. Question: What is in front of the Notre Dame Main Building?
   Answers: ['a copper statue of Christ']

3. Que

# Basic RAG

In [8]:
def add_line_breaks(text, line_length=100):
    """Add line breaks to text every specified number of characters."""
    if not text:
        return ""
    return '\n'.join(text[i:i+line_length] for i in range(0, len(text), line_length))


In [9]:
import torch
from datasets import load_dataset
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import logging

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

### RAG Vector Store

In [10]:
# This creates one context per question, which is not what we want. In the original PoisonedRAG paper they used the entire KB to query.
# def create_vectorstore_basic():

#   contexts = [item["context"] for item in squad]
#   questions = [item["question"] for item in squad]

#   text_splitter = RecursiveCharacterTextSplitter(
#       chunk_size=500,
#       chunk_overlap=100,
#   )

#   split_texts = []
#   for context in contexts:
#       split_texts.extend(text_splitter.split_text(context))

#   embeddings = HuggingFaceEmbeddings(
#       model_name="sentence-transformers/all-MiniLM-L6-v2"
#   )

#   vectorstore = FAISS.from_texts(split_texts, embeddings)

#   return vectorstore

# vectorstore = create_vector_store_basic()

In [11]:
def create_vectorstore_basic(dataset):
    # Collect ALL contexts from the entire dataset
    contexts = [item["context"] for item in dataset]

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100,
    )

    # Split ALL contexts into chunks
    split_texts = []
    for context in contexts:
        split_texts.extend(text_splitter.split_text(context))

    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

    # Create vector store from ALL split texts
    vectorstore = FAISS.from_texts(split_texts, embeddings)

    return vectorstore

### Answering Model & Prompt Setup

In [12]:
import logging
def model_setup_mistral():
    # Suppress transformers logging
    logging.getLogger('transformers').setLevel(logging.ERROR)
    # need to go to https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3 to request access
    model_name = "mistralai/Mistral-7B-Instruct-v0.3"

    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch_dtype,
        device_map="auto"
    )

    # Create text generation pipeline
    text_generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        # Suppress pipeline logging
        add_special_tokens=True,
        return_full_text=False
    )

    # Initialize LLM
    llm = HuggingFacePipeline(pipeline=text_generator)

    return llm

### RAG Inference Pipeline Basic

In [18]:
def rag_inference_pipeline_basic(llm, vectorstore, dataset, verbose=False, k=5):
    prompt_template = """Use ONLY the following context to answer the question as precisely as possible.
    If the answer is not in the context, say "I don't know based on the given context."

    Context: {context}
    Question: {question}
    Answer in a concise, direct manner: """

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | PROMPT
        | llm
        | StrOutputParser()
    )

    def process_questions(questions):
        results = []
        answers = [_['text'] for _ in squad['answers']]
        for i, (question, ground_truth) in enumerate(zip(questions, answers), 1):

            context_docs = retriever.invoke(question)
            context = "\n".join(doc.page_content for doc in context_docs)

            generated_answer = rag_chain.invoke(question)

            print(f"\n=== Question {i} ===")
            print("QUESTION:")
            print(question)
            print("\nRETRIEVED CONTEXT:")
            print(context)
            print("\nGENERATED ANSWER:")
            print(generated_answer)
            print("\nGROUND TRUTH ANSWER:")
            print(ground_truth)
            print("\n" + "="*50)

            results.append({
                "question": question,
                "context": context,
                "generated_answer": generated_answer,
                "ground_truth_answer": ground_truth
            })
        return results

    return process_questions

### RAG Inference Execution

In [16]:
vectorstore = create_vectorstore_basic(squad)
mistral = model_setup_mistral()

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

<ipython-input-12-626f0f294096>:31: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generator)


In [17]:
experiment = rag_inference_pipeline_basic(mistral, vectorstore, squad, verbose=True)

N = 5
experiment(squad['question'][:N])


=== Question 1 ===
QUESTION:
To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?

RETRIEVED CONTEXT:
of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.

GENERATED ANSWER:
 The Virgin Mary allegedly appeared to Saint Bernadette Soubirous in 1858 in Lourdes France.

GROUND TRUTH ANSWER:
['Saint Bernadette Soubirous']


=== Question 2 ===
QUESTION:
What is in front of the Notre Dame Main Building?

RETRIEVED CONTEXT:
Architectura

[{'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
  'context': 'of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.\nof prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
  'generated_answer': ' The Virgin Mary allegedly appeared to Saint Bernadette Soubirous in 1858 in Lourdes France.',
  'ground_truth_answer': ['Saint Bernadette Soubirous']},
 {'question': 'What is in front of the Notre Dame Main Building?',
  'context': 'Architecturally, the school has a Cat